In [1]:
#lambda2 Calculation
import numpy as np
import pyplume as pl

U = pl.loadmat(r"C:\Users\thanh\OneDrive - University of Texas at San Antonio\Data\NH3\NH3_U.mat")
V = pl.loadmat(r"C:\Users\thanh\OneDrive - University of Texas at San Antonio\Data\NH3\NH3_V.mat")
W = pl.loadmat(r"C:\Users\thanh\OneDrive - University of Texas at San Antonio\Data\NH3\NH3_W.mat")

In [4]:
np.shape(U)

(45, 45, 700, 540)

In [6]:
Ut = U[:,:,:,0:2]
Vt = V[:,:,:,0:2]
Wt = W[:,:,:,0:2]
np.shape(Ut)

In [27]:
def sameshape3(datau,datav,dataw):
    if (datau.shape[0] != datav.shape[0] or datau.shape[0] != dataw.shape[0]
        or datau.shape[1] != datav.shape[1] or datau.shape[1] != dataw.shape[1]
        or datau.shape[2] != datav.shape[2] or datau.shape[2] != dataw.shape[2]
        or datau.shape[3] != datav.shape[3] or datau.shape[3] != dataw.shape[3]):
        print ("Please input uvw files as same shape")
        return False
    else:
        return True
    
def transpose(x): 
    a11 = x[0,0]
    a12 = x[0,1]
    a13 = x[0,2]
    a21 = x[1,0]
    a22 = x[1,1]
    a23 = x[1,2]
    a31 = x[2,0]
    a32 = x[2,1]
    a33 = x[2,2]
    x = np.array([[a11,a21,a31],[a12,a22,a32],[a13,a23,a33]])
    return x

# def lambmatrix(S,Ome,nx,ny,nz):
#     from numpy import linalg as LA
#     lamb2 = np.zeros((nx,ny,nz),dtype=np.float32)
#     for i in range(0,nx):
#                 for j in range(0,ny):
#                     for k in range(0,nz):
#                         w = LA.eigvals(S[:,:,i,j,k].dot(S[:,:,i,j,k])+Ome[:,:,i,j,k].dot(Ome[:,:,i,j,k]))
#                         w = np.sort(w)
#                         if w[1]<0:
#                             lamb2[i,j,k] = w[1]
                            
def lambda2single(datau,datav,dataw,t,dx,dy,dz):
    from numpy import linalg as LA
    import multifluidlab
    nx,ny,nz,nt = np.shape(datau)        
        
    dudy,dudx,dudz= np.gradient(datau[:,:,:,t],dx,dy,dz)
    dvdy,dvdx,dvdz = np.gradient(datav[:,:,:,t],dx,dy,dz)
    dwdy,dwdx,dwdz = np.gradient(dataw[:,:,:,t],dx,dy,dz)

    J = np.array([[dudx,dudy,dudz],[dvdx,dvdy,dvdz],[dwdx,dwdy,dwdz]])
    JT = transpose(J)
    
    S = (J+JT)*0.5
    Ome = (J-JT)*0.5
    
    lamb2 = np.zeros((nx,ny,nz),dtype=np.float32)
    for i in range(0,nx):
                for j in range(0,ny):
                    for k in range(0,nz):
                        w = LA.eigvals(S[:,:,i,j,k].dot(S[:,:,i,j,k])+Ome[:,:,i,j,k].dot(Ome[:,:,i,j,k]))
                        w = np.sort(w)
                        if w[1]<0:
                            lamb2[i,j,k] = w[1]
    
#     lamb2 = lambmatrix(S,Ome,nx,ny,nz)
    return lamb2

def lambda2(datau,datav,dataw,dx,dy,dz):
    import numpy as np
    if (sameshape3(datau,datav,dataw) is False):
        return
    nx,ny,nz,nt = np.shape(datau)
    lamb2 = np.zeros((nx,ny,nz,nt),dtype=np.float32)
    for i in range(nt):
        percent = np.str(np.round(i/nt*100))+"%"
        print('{}\r'.format(percent), end="")
        lamb2[:,:,:,i] = lambda2single(datau,datav,dataw,i,dx,dy,dz)
    print('{}\r'.format("Lambda2 Vortex Identification Completed "), end="")
    return lamb2

In [28]:
l2 = lambda2(Ut,Vt,Wt,40,40,10)